# Linear Gaussian test  
deismic

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import sbi.simulators as simulators
import sbi.utils as utils

from torch import distributions
from sbi.inference.snpe.snpe_c import APT
import numpy as np

# use cpu by default
torch.set_default_tensor_type("torch.FloatTensor")

# seed the simulations
torch.manual_seed(0)

## Create NN instance

In [ ]:
dim, std = 3, 0.5
simulator = simulators.TwoMoonsSimulator()
a = 1
prior = distributions.Uniform(low=-a * torch.ones(simulator.parameter_dim), high=a * torch.ones(simulator.parameter_dim))

In [ ]:
true_observation = torch.tensor([[0, 0]])
ground_truth_parameters = torch.tensor([0, 0])

In [ ]:
neural_posterior = utils.get_sbi_posterior(
    model='maf',
    prior=prior,
    context=true_observation,
)

In [ ]:
# creating apt object
apt = APT(
    simulator=simulator,
    true_observation=true_observation,
    prior=prior,
    num_atoms=10,
    density_estimator=neural_posterior,
    z_score_obs=True,
    use_combined_loss=False,
    retrain_from_scratch_each_round=False,
    discard_prior_samples=False,
)

### Run inference

In [ ]:
num_rounds, num_simulations_per_round = 2, [800, 500]
apt.run_inference(
    num_rounds=num_rounds, num_simulations_per_round=num_simulations_per_round, batch_size=50
)

### Draw samples and plot hist

In [ ]:
samples = apt._neural_posterior.sample(2500)
samples = utils.tensor2numpy(samples)
figure = utils.plot_hist_marginals(
    data=samples,
    ground_truth=utils.tensor2numpy(ground_truth_parameters).reshape(-1),
    lims=simulator.parameter_plotting_limits,
)